# <font color="red"> TP n°3 - chiffrement asymétrique </font>

## <font color="green"> 1 - RSA : mise en pratique pour du chiffrement </font>

### <font color="blue"> Exercicce 1.1 (Exploitation d'un code pour chiffrer un texte selon RSA) </font>

In [1]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

rsa_obj = RSA.generate(2048)
rsa_pub = rsa_obj.publickey()

with open('mPubKey.pem','wb') as f:
    f.write(rsa_pub.exportKey('PEM'))

with open('mPrivKey.pem','wb') as fp:
    fp.write(rsa_obj.exportKey('PEM'))

In [2]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP


with open('mPubKey.pem','rb') as f:
    pubKey = RSA.importKey(f.read())
    encryptor = PKCS1_OAEP.new(pubKey)
    message_to_encrypt = 'Securité L3!'
    ciphertext = encryptor.encrypt(message_to_encrypt.encode('utf-8'))

with open("cipher.txt","wb") as fc :
    fc.write(ciphertext)

In [3]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP



with open("cipher.txt","rb") as fd :
    ciphertext = fd.read()


with open('mPrivKey.pem','rb') as fe:
    privKey = RSA.importKey(fe.read())
    decryptor = PKCS1_OAEP.new(privKey)
    decrypted_message = decryptor.decrypt(ciphertext)
    print(decrypted_message.decode('utf-8'))

Securité L3!


In [4]:
from Crypto.PublicKey import RSA

with open('mPubKey.pem', 'rb') as f:
    pubKey = RSA.importKey(f.read())
    
with open('mPrivKey.pem', 'rb') as fe:
    privKey = RSA.importKey(fe.read())
    
print(f"Public Key: (n={hex(pubKey.n)}, e={hex(pubKey.e)}))")
print(f"Private Key: (n={hex(privKey.n)}, e={hex(privKey.e)}))")

Public Key: (n=0xb3598e7f16aeddb4c516146215262f7a315e55b09998552c8a305088633993939e493f355f1576628c0952a10b21ee06181ef427628f2846cdc12dd5169910bc9178f99f18d23eab2bde089a0da847bd3dc989fb1e573a4bcd6e656e369b61e2814d8400fa9a767f57865122300a6052e14bcb1138707ae0c0c517686123e7e4409747e8bfe363144bff44d3b58eafd7d35a23f9a283879904eeb50747e4fe4bd86487da4450d419d74a80291406b62607a8d21dfb62d358109ffce06e59d7f1c16e18e0817a861c9e9e5a055f25b101adbdda65e8a75351d41bdd5c9d4413b880bf0f0328201630c148b0ac9f93da58063bae43b2c9c6e42a82e239c8ac5625, e=0x10001))
Private Key: (n=0xb3598e7f16aeddb4c516146215262f7a315e55b09998552c8a305088633993939e493f355f1576628c0952a10b21ee06181ef427628f2846cdc12dd5169910bc9178f99f18d23eab2bde089a0da847bd3dc989fb1e573a4bcd6e656e369b61e2814d8400fa9a767f57865122300a6052e14bcb1138707ae0c0c517686123e7e4409747e8bfe363144bff44d3b58eafd7d35a23f9a283879904eeb50747e4fe4bd86487da4450d419d74a80291406b62607a8d21dfb62d358109ffce06e59d7f1c16e18e0817a861c9e9e5a055f25b101adbdda65e8a75351d41bdd5

ce code affiche la clé publique et privé enregistrer dans mPrivKey.pem et mPubKeay.pem (n et e représentant repsectivement le grand nombre et l'exposant de la clé)

### <font color="blue"> Exercice 1.2 (Transfert d'une clé AES et chiffrement avec celle-ci) </font>

In [5]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import os

# Chemin de l'image
image_path = './image/kirbo.png'

# Etape 1: Génération d'une clé AES
key_aes = get_random_bytes(16)  # Génération d'une clé AES de 128 bits

# Etape 2: Chiffrement de la clé AES avec RSA
# Chargement de la clé publique RSA
with open('mPubKey.pem', 'rb') as f_pub:
    pub_key = RSA.importKey(f_pub.read())
    cipher_rsa = PKCS1_OAEP.new(pub_key)
    encrypted_aes_key = cipher_rsa.encrypt(key_aes)

# Sauvegarde de la clé AES chiffrée pour transmission
with open('encrypted_aes_key.bin', 'wb') as f_encrypted_key:
    f_encrypted_key.write(encrypted_aes_key)

# Simulation de l'étape 3: Chiffrement de l'image avec la clé AES
# Lecture de l'image
with open(image_path, 'rb') as f_image:
    image_data = f_image.read()

# Chiffrement AES de l'image
cipher_aes = AES.new(key_aes, AES.MODE_CBC)
iv = cipher_aes.iv
ciphertext = cipher_aes.encrypt(pad(image_data, AES.block_size))

# Sauvegarde du chiffré et de l'IV pour transmission
with open('encrypted_image.bin', 'wb') as f_encrypted_image:
    f_encrypted_image.write(iv + ciphertext)

# Etape 4: Déchiffrement de l'image
# Lecture de la clé AES chiffrée
with open('encrypted_aes_key.bin', 'rb') as f_encrypted_key:
    encrypted_aes_key = f_encrypted_key.read()

# Déchiffrement de la clé AES avec RSA
with open('mPrivKey.pem', 'rb') as f_priv:
    priv_key = RSA.importKey(f_priv.read())
    cipher_rsa = PKCS1_OAEP.new(priv_key)
    decrypted_aes_key = cipher_rsa.decrypt(encrypted_aes_key)

# Lecture de l'image chiffrée
with open('encrypted_image.bin', 'rb') as f_encrypted_image:
    iv = f_encrypted_image.read(16)  # AES utilise un bloc de 128 bits (16 octets)
    ciphertext = f_encrypted_image.read()

# Déchiffrement AES de l'image
cipher_aes = AES.new(decrypted_aes_key, AES.MODE_CBC, iv)
decrypted_text = unpad(cipher_aes.decrypt(ciphertext), AES.block_size)

# Affichage du résultat du déchiffrement
decrypted_text

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xb0\x00\x00\x02\xb7\x08\x06\x00\x00\x01\xcb\xc8\x93X\x00\x00\x00\tpHYs\x00\x00\x0b\x12\x00\x00\x0b\x12\x01\xd2\xdd~\xfc\x00\x06\xe5\xd2IDATx\xda\xec\x9dw\x9c]U\xd5\xf7\xbfk\x9fr\xcb\xf4\xc9$\x93\xde{!@B\x0b\x02\x02\xd2\xa5\x84*\x08H\x93\xa2\xe8\x83"\xa2>\xa8\x88\xa0(<bA\x11\x01A\xe9=\x80tC\x87\x00!@B\x0b!\xbdL\xea$S\xee\x9d{\xef){\xbd\x7f\x9c;\x93\x04\x01Q\x08\x81\xbc\xac|\xceg&\xf7\xce\xdc9{\xff\xce\xdak\xed\xdf*\xdb\xe1s\xd9\xa4\xf2\xd5\x1d\xa8\x16e\xe0\xcav\x9a?\x9f\x8d-D\x1c\xe1\x1f"\xcc7FZ\x8d1y\x11V\x18\xf8\xe6\'\xf1\xb7\xe5\xf3\xe9\xdf4\xe29<\x11\xab\xd9\xb1\xb1w\x1f\xef\x80\xfd\xf75\x95\xd9\x0cw\xdf}\x9f]\xbcxakuJ_l-\xb2\xef\xe7\xb3\xf4\x19\x93j\x8f\x9b\x8cHn\xd0\x90\x81v\xcd\xda5\x1aE\xa1\xc6q\xa4s\x16\xce\xd3\xea\xfaz+\xd0>i$\x87}>S\x9f!9t\x1f\xb6\x15a^&\x93-\xbe\xf4\xca\x8b\x1aE\x91ZkU\xad\xd5 \x08\xf4\xb0#\x8f\xb2bL\xe4\x19\xae\xd9\x94\xf7\xe1~\x0e\xc5\x7f&{\xf4c\xd7\x95i\x06/[\xcbo\xda\x9a\xa9\xb2`\x00\x11PS\x81\x9d\xf2\x08

## <font color="green"> 2 -  RSA : mise en pratique pour signer un document </font>

### <font color="blue"> Exercice 2.1 (Mise en place d'une signature avec pycryptodome) </font>

In [6]:
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA

# Fonctions de signature et de vérification inchangées
def sign_message(message, privKey):
    h = SHA256.new(message)
    signature = pkcs1_15.new(privKey).sign(h)
    return signature

def verify_signature(message, signature, pubKey):
    h = SHA256.new(message)
    try:
        pkcs1_15.new(pubKey).verify(h, signature)
        print("La signature est valide.")
    except (ValueError, TypeError):
        print("La signature n'est pas valide.")

# Chargement des clés
privKey = RSA.importKey(open('mPrivKey.pem', 'rb').read())
pubKey = RSA.importKey(open('mPubKey.pem', 'rb').read())

# Chargement et signature des données de l'image
image_path = './image/kirbo.png'
with open(image_path, 'rb') as f:
    image_data = f.read()

signature = sign_message(image_data, privKey)

# Vérification de la signature avec les données originales de l'image
print("image normale")
verify_signature(image_data, signature, pubKey)
print("\n")
# Simulation de la modification de l'image (par exemple, en ajoutant un byte)
image_data_modified = image_data + b'\x00'

# Vérification de la signature avec les données modifiées de l'image
print("image modifié")
verify_signature(image_data_modified, signature, pubKey)

image normale
La signature est valide.


image modifié
La signature n'est pas valide.
